<a href="https://colab.research.google.com/github/HedersonSantos/SmartHome/blob/main/seriesTemporais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ERROR: Could not find a version that satisfies the requirement statsmodels.tsa (from versions: none)
ERROR: No matching distribution found for statsmodels.tsa


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime



from statsmodels.tsa.stattools import acovf, acf, pacf, pacf_yw, pacf_ols
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf # para determinar (p,q)
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.tools import diff
from statsmodels.tsa.stattools import adfuller

from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.forecasting.theta import ThetaModel

from pmdarima.arima.utils import ndiffs
from pmdarima import auto_arima


from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.preprocessing import MinMaxScaler


In [32]:
!wget https://raw.githubusercontent.com/HedersonSantos/SmartHome/main/train.csv
!wget https://raw.githubusercontent.com/HedersonSantos/SmartHome/main/test.csv
!wget https://raw.githubusercontent.com/HedersonSantos/SmartHome/main/sample_submission.csv

--2022-05-17 23:42:06--  https://raw.githubusercontent.com/HedersonSantos/SmartHome/main/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 360467 (352K) [text/plain]
Saving to: ‘train.csv.1’

train.csv.1         100%[===================>] 352.02K  --.-KB/s    in 0.06s   

2022-05-17 23:42:06 (5.38 MB/s) - ‘train.csv.1’ saved [360467/360467]

--2022-05-17 23:42:06--  https://raw.githubusercontent.com/HedersonSantos/SmartHome/main/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170613 (167K) [text/plain]
Saving to: 

In [30]:
!head train.csv

Id,Date,Time,CO2_(dinning-room),CO2_room,Relative_humidity_(dinning-room),Relative_humidity_room,Lighting_(dinning-room),Lighting_room,Meteo_Rain,Meteo_Sun_dusk,Meteo_Wind,Meteo_Sun_light_in_west_facade,Meteo_Sun_light_in_east_facade,Meteo_Sun_light_in_south_facade,Meteo_Sun_irradiance,Outdoor_relative_humidity_Sensor,Day_of_the_week,Indoor_temperature_room
0,13/03/2012,11:45,216.56,221.92,39.9125,42.415,81.665,113.52,0.0,623.36,1.42625,9690.24,12604.2,95436.8,758.88,48.375,2.0,17.8275
1,13/03/2012,12:00,219.947,220.363,39.9267,42.2453,81.7413,113.605,0.0,623.211,1.592,11022.0,10787.2,95436.8,762.069,47.808,2.0,18.1207
2,13/03/2012,12:15,219.403,218.933,39.772,42.2267,81.424,113.6,0.0,622.656,1.89133,13960.5,9669.63,95398.6,766.251,47.432,2.0,18.4367
3,13/03/2012,12:30,218.613,217.045,39.776,42.0987,81.5013,113.344,0.0,622.571,1.828,18511.2,9648.13,95360.3,766.037,47.024,2.0,18.7513
4,13/03/2012,12:45,217.714,216.08,39.7757,42.0686,81.4657,113.034,0.0,622.4,2.36071,26349.0,9208.32,9535

In [51]:
df_treino = pd.read_csv('train.csv')
df_treino['data_hora'] = df_treino.loc[:,['Date', 'Time']].apply(lambda x: datetime.strptime(x['Date'] + ' ' + x['Time'], '%d/%m/%Y %H:%M'), axis=1)
df_treino = df_treino.sort_values(by='data_hora')
df_treino = df_treino.set_index('data_hora')
idx = pd.date_range(start=df_treino.index.min(), end=df_treino.index.max(),  freq='0h15min')
df = df_treino.reindex(idx)
df_treino.fillna(method='ffill', inplace=True)

In [52]:
df_treino.head(10)

,Id,Date,Time,CO2_(dinning-room),CO2_room,Relative_humidity_(dinning-room),Relative_humidity_room,Lighting_(dinning-room),Lighting_room,Meteo_Rain,Meteo_Sun_dusk,Meteo_Wind,Meteo_Sun_light_in_west_facade,Meteo_Sun_light_in_east_facade,Meteo_Sun_light_in_south_facade,Meteo_Sun_irradiance,Outdoor_relative_humidity_Sensor,Day_of_the_week,Indoor_temperature_room
data_hora,,,,,,,,,,,,,,,,,,,
2012-03-13 11:45:00,0,13/03/2012,11:45,216.560,221.920,39.9125,42.4150,81.6650,113.5200,0.0,623.360,1.42625,9690.24,12604.20,95436.8,758.880,48.3750,2.0,17.8275
2012-03-13 12:00:00,1,13/03/2012,12:00,219.947,220.363,39.9267,42.2453,81.7413,113.6050,0.0,623.211,1.59200,11022.00,10787.20,95436.8,762.069,47.8080,2.0,18.1207
2012-03-13 12:15:00,2,13/03/2012,12:15,219.403,218.933,39.7720,42.2267,81.4240,113.6000,0.0,622.656,1.89133,13960.50,9669.63,95398.6,766.251,47.4320,2.0,18.4367
2012-03-13 12:30:00,3,13/03/2012,12:30,218.613,217.045,39.7760,42.0987,81.5013,113.3440,0.0,622.571,1.82800,18511.20,9648.13,95360.3,766.037,47.0240,2.0,18.7513
2012-03-13 12:45:00,4,13/03/2012,12:45,217.714,216.080,39.7757,42.0686,81.4657,113.0340,0.0,622.400,2.36071,26349.00,9208.32,95354.9,762.743,45.4743,2.0,19.0414
2012-03-13 13:00:00,5,13/03/2012,13:00,218.389,216.011,39.8107,42.0347,81.2427,113.6000,0.0,622.400,2.06333,36511.70,8694.78,95354.9,752.683,45.5467,2.0,19.2773
2012-03-13 13:15:00,6,13/03/2012,13:15,218.155,215.829,39.8613,41.9947,81.3600,112.9810,0.0,622.400,2.23067,49288.50,8501.25,95354.9,740.949,44.9680,2.0,19.5367
2012-03-13 13:30:00,7,13/03/2012,13:30,216.661,215.296,39.8240,41.9947,81.0240,111.2590,0.0,621.803,1.92400,62253.70,8687.62,95174.7,721.109,45.1813,2.0,19.7307
2012-03-13 13:45:00,8,13/03/2012,13:45,217.461,214.549,39.9587,42.1253,81.1040,83.3547,0.0,620.651,2.51067,71453.40,8888.32,91848.7,699.392,45.4160,2.0,19.9693
